In [1]:
from src.models.level_kv_div import binaryTrainer, utils, network
from torch.utils.data import DataLoader


import torch

In [ ]:
# train_dataset = binaryTrainer.SparseDataset('../data/processed/sample/train.csv', '../data/processed/sample/train.level.grading', balanced=True, random_neg=False)
# dev_dataset = binaryTrainer.SparseDataset('../data/processed/sample/dev.csv', '../data/processed/sample/dev.level.grading', balanced=True, random_neg=False)
train_dataset = binaryTrainer.LargeSparseDataset('../data/processed/sample/train.csv', 100, balanced=True, random_neg=True)
dev_dataset = binaryTrainer.LargeSparseDataset('../data/processed/sample/dev.csv', 100, balanced=True, random_neg=False)


In [ ]:
model = network.SiameseNet(network.EmbeddingNet())
learning_rate = 4e-3
batch_size = 32768
num_epoches = 100

verbose = True
device = 'cuda' if torch.cuda.is_available() else 'cpu'
weight_decay = 1e-6
log_epoch = 3

In [ ]:
# optimizer
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(
        nd in n for nd in no_decay) and p.requires_grad], 'weight_decay': weight_decay},
    {'params': [p for n, p in model.named_parameters() if any(
        nd in n for nd in no_decay) and p.requires_grad], 'weight_decay': 0.0}
]

optimizer = torch.optim.AdamW(params=optimizer_grouped_parameters, lr=learning_rate)

train_loader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size, pin_memory=True)
dev_loader = DataLoader(dev_dataset, shuffle=False, batch_size=batch_size, pin_memory=True)
model = model.to(device)

In [ ]:
best_avg_val_kl_loss, best_model, model = binaryTrainer.train_with_eval(train_loader, dev_loader, model, optimizer, num_epoches, log_epoch, verbose, device)

In [ ]:
best_avg_val_kl_loss

In [ ]:
criterion = binaryTrainer.TriMarginLoss(1, 3, 4, 6, 'mean')

In [ ]:
output1, output2 = model(train_dataset[0][0].unsqueeze(0).to(device), train_dataset[0][1].unsqueeze(0).to(device))
loss, dist = criterion.forward(output1, output2, train_dataset[0][2])
print(loss.item(), dist.item())

In [ ]:
((output1 - output2) ** 2).sum()

In [ ]:
test_dataset = binaryTrainer.LargeSparseDataset('../data/processed/sample/train.csv', 100, balanced=True, random_neg=False)

test_dataloader = DataLoader(test_dataset, shuffle=True, batch_size=1024, pin_memory=True)

In [ ]:
test_loss, (test_acc, test_pred, test_gold, test_dist) = binaryTrainer.val_one_epoch(test_dataloader, best_model.to(device), device)

In [ ]:
test_acc

In [ ]:
# torch.save(best_model, 'margin_model.pth.tar')